In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizVert_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,137329.22,0.68,0.79,-0.19,0.14,0.06,-0.01,-0.00
3,0.68,31676.88,-0.65,-0.08,0.04,-0.01,0.00,-0.00
4,0.79,-0.65,8240.65,-0.04,0.02,0.04,0.00,0.00
5,-0.19,-0.08,-0.04,1870.21,-0.02,-0.02,-0.00,-0.00
6,0.14,0.04,0.02,-0.02,362.22,0.00,0.00,0.00
7,0.06,-0.01,0.04,-0.02,0.00,62.59,-0.00,0.00
8,-0.01,0.00,0.00,-0.00,0.00,-0.00,9.08,0.00
9,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.85


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00001,0.00002,-0.00001,0.00002,0.00002,-0.00001,-0.00001
3,0.00001,1.00000,-0.00004,-0.00001,0.00001,-0.00000,0.00001,-0.00002
4,0.00002,-0.00004,1.00000,-0.00001,0.00001,0.00006,0.00000,0.00000
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00002,-0.00005,-0.00000,-0.00000
6,0.00002,0.00001,0.00001,-0.00002,1.00000,0.00002,0.00001,0.00001
7,0.00002,-0.00000,0.00006,-0.00005,0.00002,1.00000,-0.00002,0.00006
8,-0.00001,0.00001,0.00000,-0.00000,0.00001,-0.00002,1.00000,0.00002
9,-0.00001,-0.00002,0.00000,-0.00000,0.00001,0.00006,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.7764809025566661

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.36028585e+09 1.22991979e+08 8.08572844e+06 3.99832099e+05
 1.40709478e+04 3.73876205e+02 5.90761167e+00 6.79323229e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999359,-0.034711,-0.008512,-0.001927,-0.000372,-0.000064,-0.000009,-8.900827e-07
1,0.034256,0.998272,-0.046689,-0.009797,-0.001888,-0.000327,-0.000047,-4.545506e-06
2,0.009993,0.045829,0.997695,-0.048222,-0.008779,-0.001509,-0.000220,-2.056388e-05
3,0.002716,0.011804,0.047184,0.997493,-0.050627,-0.008358,-0.001210,-1.133620e-04
4,0.000654,0.002841,0.010937,0.049534,0.996896,-0.059564,-0.008301,-7.770174e-04
5,0.000150,0.000653,0.002506,0.011081,0.058276,0.995458,-0.074116,-6.716531e-03
6,0.000033,0.000142,0.000545,0.002403,0.012403,0.072533,0.992847,-9.398544e-02
7,0.000006,0.000027,0.000103,0.000453,0.002336,0.013516,0.093224,9.955506e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006407843887590303,
 0.0017279809791096445,
 0.002305098623321511,
 0.0025068828230332407,
 0.003104408349143939,
 0.004542262058150781,
 0.007152718232717348,
 0.004449394617889624]